# Class Metaprogramming

Class metaprogramming is the art of creating or customizing classes at runtime. 

In [7]:
class MyMixin:
    pass

class MySuperClass:
    pass


Consider this simple class:

```python
class MyClass(MySuperClass, MyMixin):
    x = 42

    def x2(self):
        return self.x * 2
```

Using the `type` constructor, you can create `MyClass` at runtime with this code: 

In [8]:
MyClass = type(
    'MyClass',
    (MySuperClass, MyMixin),
    {'x': 42, 'x2': lambda self: self.x * 2}
)

When Python reads a `class` statement, it calls `type` to build the class object with these parameters:

- `name`
- `bases`
- `dict`

The `type` class is a *metaclass*: a class that builds classes.

In [9]:
print(type(7))
print(type(int))
print(type(OSError))

<class 'int'>
<class 'type'>
<class 'type'>


## Introducing `__init_subclass__`

In [10]:
from collections.abc import Callable  
from typing import Any, NoReturn, get_type_hints


class Field:
    # This is a minimal `Callable` type hint; 
    # the parameter type and return type for `constructor` are both implicity `Any`. 
    def __init__(self, name: str, constructor: Callable) -> None:  
        if not callable(constructor) or constructor is type(None): 
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor


    def __set__(self, instance: Any, value: Any) -> None:
        # `...`, the Ellipsis built-in object
        if value is ...:  
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)  
            except (TypeError, ValueError) as e:  
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value  

The `@classmethod` is never used with `__init_subclass__`, because the `__new__` special method behaves as a class method even without `@classmethod`. 

The first argument that Python passes to `__init_subclass__` is a class. 

However, it is never the class where `__init_subclass__` is implemented: it is a newly defined subclass of that class.  
[`__init_subclass__` documentation](https://fpy.li/24-8)

In [11]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)
    

    # `__init_subclass__` is called when a subclass of the current class is defined.
    # It gets that new subclass as its first argument ——
    # which is why we named the argument `subclass`, instead of the usual `cls`. 
    def __init_subclass__(subclass) -> None:  
        super().__init_subclass__()           

        # Iterate over field `name` and `constructor`, 
        # creating an attribut on `subclass` with that 
        # `name` bound to a `Field` descriptor parameterized with `name` and `constructor`. 
        for name, constructor in subclass._fields().items():   
            setattr(subclass, name, Field(name, constructor))  


    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():             
            value = kwargs.pop(name, ...)     
            # This `setattr` call triggers `Checked.__setattr__`
            setattr(self, name, value)       
        if kwargs:                            
            self.__flag_unknown_attrs(*kwargs)
    

    # Intercept all attempts to set an instance attribute. 
    # This is needed to prevent setting an unknown attribute. 
    def __setattr__(self, name: str, value: Any) -> None: 

        # If the attribute `name` is known, fetch the corresponding `descriptor`. 
        if name in self._fields():              
            cls = self.__class__
            descriptor = getattr(cls, name)
            # Usually we don't need to call `__set__` explicity. 
            # It was necessary in this case because `__setattr__` intercepts all attempts 
            # to set an attribute on the instance, 
            # including in the presence of an overriding descriptor such as `Field`. 
            descriptor.__set__(self, value)     
        else:                                 
            self.__flag_unknown_attrs(name)


    # Build a helpful error message listing all unexpected arguments. 
    def __flag_unknown_attrs(self, *names: str) -> NoReturn:  
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')


    # Create a `dict` from the attributes of a `Movie` object. 
    def _asdict(self) -> dict[str, Any]:  
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }


    def __repr__(self) -> str:  
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

### Why `__init_subclass__` Cannot Configure `__slots__`

The `__slots__` attribute is only effective if it is one of the entries in the class namespace passed to `type.__new__`.

Adding `__slots__` to an existing class has no effect. 

Python invokes `__init_subclass__` only after the class is built——by then it's too late to configure `__slots__`. 
[PEP 487](https://fpy.li/pep487)

In [12]:
class Movie(Checked):
    title: str
    year: int
    box_office: float

In [13]:
moive = Movie(title="Well", year=1984, box_office=127)
print(moive.title, moive, sep='\n')

Well
Movie(title='Well', year=1984, box_office=127.0)


## [Next Section](./ClassesDecorator.ipynb)